In [1]:
import pandas as pd
import seaborn as sns
from keras.models import Sequential
from keras.layers import Dense, Dropout
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
from sklearn.metrics import classification_report

In [7]:
#df = pd.read_csv("C:\dev\datasets\iot23_combined.csv")
#df = pd.read_csv(r"C:\Users\Bas-Study\data\iot23_com\iot23_combined2.csv")
df = pd.read_csv(r"C:\Users\Bas-Study\data\df_combined.csv")


In [8]:
df

,Unnamed: 0,ts,id.orig_h,id.orig_p,id.resp_h,id.resp_p,duration,orig_bytes,resp_bytes,history,...,conn_state_SF,conn_state_SH,conn_state_SHR,service_-,service_dhcp,service_dns,service_http,service_irc,service_ssh,service_ssl
0,0,1.525880e+09,192.168.100.103,60905.0,131.174.215.147,23.0,2.998796,0.0,0.0,S,...,0,0,0,1,0,0,0,0,0,0
1,1,1.525880e+09,192.168.100.103,44301.0,91.42.47.63,23.0,0.000000,0.0,0.0,S,...,0,0,0,1,0,0,0,0,0,0
2,2,1.525880e+09,192.168.100.103,50244.0,120.210.108.200,23.0,0.000000,0.0,0.0,S,...,0,0,0,1,0,0,0,0,0,0
3,3,1.525880e+09,192.168.100.103,34243.0,147.7.65.203,49560.0,2.998804,0.0,0.0,S,...,0,0,0,1,0,0,0,0,0,0
4,4,1.525880e+09,192.168.100.103,34840.0,145.164.35.6,21288.0,0.000000,0.0,0.0,S,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1446616,122,1.562165e+09,0.0.0.0,68.0,255.255.255.255,67.0,-1.000000,-1.0,-1.0,D,...,0,0,0,0,1,0,0,0,0,0
1446617,123,1.562165e+09,::,143.0,ff02::16,0.0,-1.000000,-1.0,-1.0,-,...,0,0,0,1,0,0,0,0,0,0
1446618,124,1.562165e+09,::,135.0,ff02::1:ffd5:cdf,136.0,-1.000000,-1.0,-1.0,-,...,0,0,0,1,0,0,0,0,0,0
1446619,125,1.562165e+09,fe80::5bcc:698e:39d5:cdf,143.0,ff02::16,0.0,-1.000000,-1.0,-1.0,-,...,0,0,0,1,0,0,0,0,0,0


In [9]:
df.loc[(df.label == '-   Benign   -'), 'label'] = 'Benign'
df['label'].value_counts()

PartOfAHorizontalPortScan     825939
Okiru                         262690
Benign                        199756
DDoS                          138777
C&C                            15100
Attack                          3915
C&C-HeartBeat                    349
C&C-FileDownload                  43
C&C-Torii                         30
FileDownload                      13
C&C-HeartBeat-FileDownload         8
C&C-Mirai                          1
Name: label, dtype: int64

In [36]:
df['id.orig_h'].value_counts().head(20)

192.168.1.197               204393
192.168.1.198               199987
192.168.100.111             199935
192.168.1.195               123106
192.168.100.103             100647
192.168.1.194                99999
192.168.1.196                99981
192.168.1.193                99973
192.168.100.108              99924
192.168.1.200                99915
192.168.2.5                  99159
192.168.100.113              13679
192.168.2.3                    979
192.168.1.132                  449
192.168.1.199                  232
0.0.0.0                        164
192.168.100.1                  154
fe80::5bcc:698e:39d5:cdf        63
192.168.2.1                     60
169.254.15.115                  51
Name: id.orig_h, dtype: int64

In [45]:
#df_node = df[df['id.orig_h']=='192.168.100.111']
df_node = df[df['id.orig_h']=='192.168.1.198']

In [47]:
df_node.shape
df_node['label'].value_counts()

Okiru                        112980
PartOfAHorizontalPortScan     56835
Benign                        30016
C&C-HeartBeat                   114
C&C                              29
C&C-FileDownload                 13
Name: label, dtype: int64

In [48]:
def drop_columns(df, columns):
    temp = df.copy()
    for column in columns:
        temp = temp[temp['label'] != column]
    return temp

columns_to_drop = ["C&C",
                    "Attack",
                    "C&C-HeartBeat",
                    "C&C-FileDownload",
                    "C&C-Torii",
                    "FileDownload",
                    "C&C-HeartBeat-FileDownload",
                    "C&C-Mirai"]

df_node = drop_columns(df_node, columns_to_drop)
df_node['label'].value_counts()

Okiru                        112980
PartOfAHorizontalPortScan     56835
Benign                        30016
Name: label, dtype: int64

In [49]:
pd.options.display.max_rows = 300
pd.options.display.max_columns = 300

In [50]:
df_node = df_node.drop(columns=['Unnamed: 0', 'id.orig_h'])

In [51]:
x = df_node.loc[:, df_node.columns != 'label'].to_numpy()
df_y = df_node.loc[:, df_node.columns == 'label']
df_y = pd.get_dummies(df_y, columns=['label'])
y = df_y.to_numpy()
df_y.columns


Index(['label_Benign', 'label_Okiru', 'label_PartOfAHorizontalPortScan'], dtype='object')

In [26]:
scaler = MinMaxScaler()

x_scaled = scaler.fit_transform(x)

In [15]:
df_node['label'].value_counts()

DDoS      24959
Okiru     24958
Benign     1068
Name: label, dtype: int64

In [16]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = .2, random_state=42)

In [19]:
#6  labels

#25 features

model = Sequential()
# model.add(Dense(25, input_dim=x.shape[1], activation='elu'))
# model.add(Dense(18, activation='elu'))
# model.add(Dense(12, activation='elu'))
# model.add(Dense(y.shape[1], activation='sigmoid'))

model.add(Dense(2000, activation='relu',input_dim=x_train.shape[1]))
model.add(Dense(1500, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(800,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(400,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(150,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(y_train.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 2000)              52000     
                                                                 
 dense_6 (Dense)             (None, 1500)              3001500   
                                                                 
 dropout_4 (Dropout)         (None, 1500)              0         
                                                                 
 dense_7 (Dense)             (None, 800)               1200800   
                                                                 
 dropout_5 (Dropout)         (None, 800)               0         
                                                                 
 dense_8 (Dense)             (None, 400)               320400    
                                                                 
 dropout_6 (Dropout)         (None, 400)              

In [20]:
model.fit(x_train, y_train, epochs=5, batch_size=10)

Epoch 1/5
1440/4079 [=========>....................] - ETA: 3:11 - loss: 1167601.1250 - accuracy: 0.4828 ETA: 3:11 - loss: 1169225.0000 - accuracy: 0.

KeyboardInterrupt: 

In [17]:
model.fit(x_train, y_train, epochs=5, batch_size=72, validation_data=(x_test, y_test), shuffle=False)

Epoch 1/5
2271/2271 [==============================] - 6s 3ms/step - loss: 0.5151 - accuracy: 0.8437 - val_loss: 0.5199 - val_accuracy: 0.8418
Epoch 2/5
2271/2271 [==============================] - 5s 2ms/step - loss: 0.5152 - accuracy: 0.8437 - val_loss: 0.5199 - val_accuracy: 0.8418
Epoch 3/5
2271/2271 [==============================] - 6s 3ms/step - loss: 0.5152 - accuracy: 0.8437 - val_loss: 0.5199 - val_accuracy: 0.8418
Epoch 4/5
2271/2271 [==============================] - 7s 3ms/step - loss: 0.5152 - accuracy: 0.8437 - val_loss: 0.5199 - val_accuracy: 0.8418
Epoch 5/5
2271/2271 [==============================] - 4s 2ms/step - loss: 0.5152 - accuracy: 0.8437 - val_loss: 0.5199 - val_accuracy: 0.8418


In [20]:
model.predict(x_test)

array([[0.03954402, 0.4491907 , 0.44393042],
       [0.03954402, 0.4491907 , 0.44393042],
       [0.03954402, 0.4491907 , 0.44393042],
       ...,
       [0.03954402, 0.4491907 , 0.44393042],
       [0.03954402, 0.4491907 , 0.44393042],
       [0.03954402, 0.4491907 , 0.44393042]], dtype=float32)

In [21]:
preds = np.argmax(model.predict(x_test), axis = 1)
num_correct = 0

for i in range(0, len(preds)):
    if preds[i] == np.argmax(y_test[i]):
        num_correct+=1

num_correct/len(preds)

0.4881827988624105

In [22]:
a = np.argmax(y_test, axis=1)
b = np.argmax(model.predict(x_test), axis=1)
print(classification_report(a, b))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       210
           1       0.49      1.00      0.66      4978
           2       0.00      0.00      0.00      5009

    accuracy                           0.49     10197
   macro avg       0.16      0.33      0.22     10197
weighted avg       0.24      0.49      0.32     10197



C:\Users\Bas-Study\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Bas-Study\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Bas-Study\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
model.save("/Users/kjwdamme/PythonProjects/datasets/first_model_scaled.h5")

In [ ]:
preds.shape

(39998,)

In [ ]:
np.unique(preds, return_counts=True)

(array([4], dtype=int64), array([39998], dtype=int64))

In [ ]:
temp = np.transpose(y_test)


In [ ]:
temp_df = pd.DataFrame(y_test)

In [ ]:
temp_df[4].value_counts()

1    22584
0    17414
Name: 4, dtype: int64